In [10]:
import os
import pandas as pd
import numpy as np
import re
import string
import emoji
import nltk

# Download necessary NLTK resources (first time only)
nltk.download('stopwords')
nltk.download('punkt')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


# Prepare stopwords
stop_words = set(stopwords.words('english'))

print("Libraries imported successfully.")



Libraries imported successfully.


[nltk_data] Downloading package stopwords to C:\Users\TUF
[nltk_data]     GAMING\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\TUF
[nltk_data]     GAMING\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Replace with your dataset path
df = pd.read_csv(r'E:saas-idea-validator\data\raw\raw_marged\merged_output.csv')

# Show first 10 rows
df.head(10)


,post_id,subreddit,title,text,author,created_utc,num_comments,upvotes,upvote_ratio,post_age_days,post_sentiment,avg_comment_sentiment,post_recency,validation_score,label,source_url,source_type,avg_comment_recency
0,1oizkq8,SaaS,"Nobody is talking about life after launching, ...","Nobody is talking about life after launching, ...",mohamednagm,2025-10-29T09:06:53Z,5,4,1.00,0,0.001465,0.563208,0.999100,40.65,neutral,https://reddit.com/r/SaaS/comments/1oizkq8/nob...,NEW_POSTS,NaN
1,1oizjpg,microsaas,Time for a quick progress,Time for a quick progress October is nearly go...,ShallotNo9818,2025-10-29T09:05:00Z,4,2,1.00,0,0.000482,0.753155,0.999055,40.78,neutral,https://reddit.com/r/microsaas/comments/1oizjp...,NEW_POSTS,NaN
2,1oiz8m8,SaaS,I built a SaaS for passive income… now I’m mor...,I built a SaaS for passive income… now I’m mor...,William45623,2025-10-29T08:44:49Z,8,7,0.89,0,0.008917,0.717628,0.998588,36.95,bad,https://reddit.com/r/SaaS/comments/1oiz8m8/i_b...,NEW_POSTS,NaN
3,1oiyulj,SaaS,How did you went from 10 to 100 paying customers?,How did you went from 10 to 100 paying custome...,Majestic-Context-290,2025-10-29T08:18:13Z,5,7,0.90,0,0.012576,0.271430,0.997972,37.11,bad,https://reddit.com/r/SaaS/comments/1oiyulj/how...,NEW_POSTS,NaN
4,1oiydve,SaaS,Anyone ever regret selling too much equity too...,Anyone ever regret selling too much equity too...,page2james,2025-10-29T07:46:21Z,7,1,0.67,0,0.009597,0.284256,0.997236,27.98,bad,https://reddit.com/r/SaaS/comments/1oiydve/any...,NEW_POSTS,NaN
5,1oiy6p7,microsaas,After 3 months I finally got my first paying u...,After 3 months I finally got my first paying u...,Small_Efficiency626,2025-10-29T07:32:36Z,4,2,0.75,0,0.064827,0.719796,0.996917,34.79,bad,https://reddit.com/r/microsaas/comments/1oiy6p...,NEW_POSTS,NaN
6,1oixzkp,Entrepreneur,What are some of the weirdest ways people have...,What are some of the weirdest ways people have...,aureliathepanda,2025-10-29T07:18:41Z,28,7,0.82,0,0.004708,0.368361,0.996594,33.62,bad,https://reddit.com/r/Entrepreneur/comments/1oi...,NEW_POSTS,NaN
7,1oixqvu,Entrepreneur,Business Advice,Business Advice I'm exploring a D2C fintech bu...,nothingsusphere,2025-10-29T07:02:16Z,9,6,1.00,0,0.968228,0.333799,0.996214,97.46,good,https://reddit.com/r/Entrepreneur/comments/1oi...,NEW_POSTS,NaN
8,1oixo6z,Entrepreneur,We are Budgeting for AI in 2026- which tools a...,We are Budgeting for AI in 2026- which tools a...,Aurora_Evana,2025-10-29T06:57:35Z,7,21,0.93,0,0.598869,0.426221,0.996105,73.03,good,https://reddit.com/r/Entrepreneur/comments/1oi...,NEW_POSTS,NaN
9,1oixj99,SaaS,I watched a $2M Series A startup almost die be...,I watched a $2M Series A startup almost die be...,Far_Flamingo5333,2025-10-29T06:48:35Z,20,8,0.79,0,0.130740,0.345931,0.995897,39.87,bad,https://reddit.com/r/SaaS/comments/1oixj99/i_w...,NEW_POSTS,NaN


In [ ]:
columns_to_keep = [
    'title', 'text', 'post_sentiment', 'avg_comment_sentiment',
    'num_comments', 'upvotes', 'upvote_ratio', 'label'
]

# Keep only existing ones (some may be missing)
df = df[[c for c in columns_to_keep if c in df.columns]].copy()

print("Columns kept:", df.columns.tolist())



df.head(10)


Columns kept: ['title', 'text', 'post_sentiment', 'avg_comment_sentiment', 'num_comments', 'upvotes', 'upvote_ratio', 'label']


,title,text,post_sentiment,avg_comment_sentiment,num_comments,upvotes,upvote_ratio,label
0,"Nobody is talking about life after launching, ...","Nobody is talking about life after launching, ...",0.001465,0.563208,5,4,1.00,neutral
1,Time for a quick progress,Time for a quick progress October is nearly go...,0.000482,0.753155,4,2,1.00,neutral
2,I built a SaaS for passive income… now I’m mor...,I built a SaaS for passive income… now I’m mor...,0.008917,0.717628,8,7,0.89,bad
3,How did you went from 10 to 100 paying customers?,How did you went from 10 to 100 paying custome...,0.012576,0.271430,5,7,0.90,bad
4,Anyone ever regret selling too much equity too...,Anyone ever regret selling too much equity too...,0.009597,0.284256,7,1,0.67,bad
5,After 3 months I finally got my first paying u...,After 3 months I finally got my first paying u...,0.064827,0.719796,4,2,0.75,bad
6,What are some of the weirdest ways people have...,What are some of the weirdest ways people have...,0.004708,0.368361,28,7,0.82,bad
7,Business Advice,Business Advice I'm exploring a D2C fintech bu...,0.968228,0.333799,9,6,1.00,good
8,We are Budgeting for AI in 2026- which tools a...,We are Budgeting for AI in 2026- which tools a...,0.598869,0.426221,7,21,0.93,good
9,I watched a $2M Series A startup almost die be...,I watched a $2M Series A startup almost die be...,0.130740,0.345931,20,8,0.79,bad


In [ ]:
print("Missing values per column:\n", df.isnull().sum())

# Drop rows with no label (since supervised learning needs labels)
if 'label' in df.columns:
    df = df.dropna(subset=['label'])

# Fill numeric NaNs with 0
num_cols = ['post_sentiment', 'avg_comment_sentiment', 'num_comments', 'upvotes', 'upvote_ratio']
for col in num_cols:
    if col in df.columns:
        df[col] = df[col].fillna(0)

# Fill text NaNs with empty strings
for col in ['title', 'text']:
    if col in df.columns:
        df[col] = df[col].fillna('')

print("After handling missing values, shape:", df.shape)
df.head(5)


Missing values per column:
 title                       0
text                        0
post_sentiment           3956
avg_comment_sentiment    3956
num_comments                0
upvotes                     0
upvote_ratio                0
label                       0
dtype: int64
After handling missing values, shape: (5019, 8)


,title,text,post_sentiment,avg_comment_sentiment,num_comments,upvotes,upvote_ratio,label
0,"Nobody is talking about life after launching, ...","Nobody is talking about life after launching, ...",0.001465,0.563208,5,4,1.00,neutral
1,Time for a quick progress,Time for a quick progress October is nearly go...,0.000482,0.753155,4,2,1.00,neutral
2,I built a SaaS for passive income… now I’m mor...,I built a SaaS for passive income… now I’m mor...,0.008917,0.717628,8,7,0.89,bad
3,How did you went from 10 to 100 paying customers?,How did you went from 10 to 100 paying custome...,0.012576,0.271430,5,7,0.90,bad
4,Anyone ever regret selling too much equity too...,Anyone ever regret selling too much equity too...,0.009597,0.284256,7,1,0.67,bad


In [ ]:
df['text_no_url'] = df['text'].apply(lambda x: re.sub(r'http\S+|www\S+', '', str(x)))

df[['text', 'text_no_url']].head(10)



,text,text_no_url
0,"Nobody is talking about life after launching, ...","Nobody is talking about life after launching, ..."
1,Time for a quick progress October is nearly go...,Time for a quick progress October is nearly go...
2,I built a SaaS for passive income… now I’m mor...,I built a SaaS for passive income… now I’m mor...
3,How did you went from 10 to 100 paying custome...,How did you went from 10 to 100 paying custome...
4,Anyone ever regret selling too much equity too...,Anyone ever regret selling too much equity too...
5,After 3 months I finally got my first paying u...,After 3 months I finally got my first paying u...
6,What are some of the weirdest ways people have...,What are some of the weirdest ways people have...
7,Business Advice I'm exploring a D2C fintech bu...,Business Advice I'm exploring a D2C fintech bu...
8,We are Budgeting for AI in 2026- which tools a...,We are Budgeting for AI in 2026- which tools a...
9,I watched a $2M Series A startup almost die be...,I watched a $2M Series A startup almost die be...


In [ ]:
df['text_no_emoji'] = df['text_no_url'].apply(lambda x: emoji.replace_emoji(x, replace=''))

df[['text_no_url', 'text_no_emoji']].head(10)


,text_no_url,text_no_emoji
0,"Nobody is talking about life after launching, ...","Nobody is talking about life after launching, ..."
1,Time for a quick progress October is nearly go...,Time for a quick progress October is nearly go...
2,I built a SaaS for passive income… now I’m mor...,I built a SaaS for passive income… now I’m mor...
3,How did you went from 10 to 100 paying custome...,How did you went from 10 to 100 paying custome...
4,Anyone ever regret selling too much equity too...,Anyone ever regret selling too much equity too...
5,After 3 months I finally got my first paying u...,After 3 months I finally got my first paying u...
6,What are some of the weirdest ways people have...,What are some of the weirdest ways people have...
7,Business Advice I'm exploring a D2C fintech bu...,Business Advice I'm exploring a D2C fintech bu...
8,We are Budgeting for AI in 2026- which tools a...,We are Budgeting for AI in 2026- which tools a...
9,I watched a $2M Series A startup almost die be...,I watched a $2M Series A startup almost die be...


In [ ]:
df['text_no_punct'] = df['text_no_emoji'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

df[['text_no_emoji', 'text_no_punct']].head(10)


,text_no_emoji,text_no_punct
0,"Nobody is talking about life after launching, ...",Nobody is talking about life after launching b...
1,Time for a quick progress October is nearly go...,Time for a quick progress October is nearly go...
2,I built a SaaS for passive income… now I’m mor...,I built a SaaS for passive income… now I’m mor...
3,How did you went from 10 to 100 paying custome...,How did you went from 10 to 100 paying custome...
4,Anyone ever regret selling too much equity too...,Anyone ever regret selling too much equity too...
5,After 3 months I finally got my first paying u...,After 3 months I finally got my first paying u...
6,What are some of the weirdest ways people have...,What are some of the weirdest ways people have...
7,Business Advice I'm exploring a D2C fintech bu...,Business Advice Im exploring a D2C fintech bus...
8,We are Budgeting for AI in 2026- which tools a...,We are Budgeting for AI in 2026 which tools ar...
9,I watched a $2M Series A startup almost die be...,I watched a 2M Series A startup almost die bec...


In [ ]:
df['text_no_num'] = df['text_no_punct'].apply(lambda x: re.sub(r'\d+', '', x))

df[['text_no_punct', 'text_no_num']].head(10)


,text_no_punct,text_no_num
0,Nobody is talking about life after launching b...,Nobody is talking about life after launching b...
1,Time for a quick progress October is nearly go...,Time for a quick progress October is nearly go...
2,I built a SaaS for passive income… now I’m mor...,I built a SaaS for passive income… now I’m mor...
3,How did you went from 10 to 100 paying custome...,How did you went from to paying customers We...
4,Anyone ever regret selling too much equity too...,Anyone ever regret selling too much equity too...
5,After 3 months I finally got my first paying u...,After months I finally got my first paying us...
6,What are some of the weirdest ways people have...,What are some of the weirdest ways people have...
7,Business Advice Im exploring a D2C fintech bus...,Business Advice Im exploring a DC fintech busi...
8,We are Budgeting for AI in 2026 which tools ar...,We are Budgeting for AI in which tools are wo...
9,I watched a 2M Series A startup almost die bec...,I watched a M Series A startup almost die beca...


In [ ]:
df['text_no_special'] = df['text_no_num'].apply(lambda x: re.sub(r'[^A-Za-z\s]', '', x))

df[['text_no_num', 'text_no_special']].head(10)


,text_no_num,text_no_special
0,Nobody is talking about life after launching b...,Nobody is talking about life after launching b...
1,Time for a quick progress October is nearly go...,Time for a quick progress October is nearly go...
2,I built a SaaS for passive income… now I’m mor...,I built a SaaS for passive income now Im more ...
3,How did you went from to paying customers We...,How did you went from to paying customers We...
4,Anyone ever regret selling too much equity too...,Anyone ever regret selling too much equity too...
5,After months I finally got my first paying us...,After months I finally got my first paying us...
6,What are some of the weirdest ways people have...,What are some of the weirdest ways people have...
7,Business Advice Im exploring a DC fintech busi...,Business Advice Im exploring a DC fintech busi...
8,We are Budgeting for AI in which tools are wo...,We are Budgeting for AI in which tools are wo...
9,I watched a M Series A startup almost die beca...,I watched a M Series A startup almost die beca...


In [ ]:
df['text_cleaned'] = df['text_no_special'].apply(lambda x: re.sub(r'\s+', ' ', x.lower()).strip())

df[['text_no_special', 'text_cleaned']].head(10)


,text_no_special,text_cleaned
0,Nobody is talking about life after launching b...,nobody is talking about life after launching b...
1,Time for a quick progress October is nearly go...,time for a quick progress october is nearly go...
2,I built a SaaS for passive income now Im more ...,i built a saas for passive income now im more ...
3,How did you went from to paying customers We...,how did you went from to paying customers were...
4,Anyone ever regret selling too much equity too...,anyone ever regret selling too much equity too...
5,After months I finally got my first paying us...,after months i finally got my first paying use...
6,What are some of the weirdest ways people have...,what are some of the weirdest ways people have...
7,Business Advice Im exploring a DC fintech busi...,business advice im exploring a dc fintech busi...
8,We are Budgeting for AI in which tools are wo...,we are budgeting for ai in which tools are wor...
9,I watched a M Series A startup almost die beca...,i watched a m series a startup almost die beca...


In [ ]:
df['text_final'] = df['text_cleaned'].apply(lambda x: ' '.join(word for word in x.split() if word not in stop_words))

df[['text_cleaned', 'text_final']].head(10)


,text_cleaned,text_final
0,nobody is talking about life after launching b...,nobody talking life launching everyone glorifi...
1,time for a quick progress october is nearly go...,time quick progress october nearly gone share ...
2,i built a saas for passive income now im more ...,built saas passive income im active ever dream...
3,how did you went from to paying customers were...,went paying customers building growthos helpin...
4,anyone ever regret selling too much equity too...,anyone ever regret selling much equity early v...
5,after months i finally got my first paying use...,months finally got first paying user today bui...
6,what are some of the weirdest ways people have...,weirdest ways people made decent amount income...
7,business advice im exploring a dc fintech busi...,business advice im exploring dc fintech busine...
8,we are budgeting for ai in which tools are wor...,budgeting ai tools worth hype bb business plan...
9,i watched a m series a startup almost die beca...,watched series startup almost die nobody could...


In [ ]:
df.head(10)

,title,text,post_sentiment,avg_comment_sentiment,num_comments,upvotes,upvote_ratio,label,text_no_url,text_no_emoji,text_no_punct,text_no_num,text_no_special,text_cleaned,text_final
0,"Nobody is talking about life after launching, ...","Nobody is talking about life after launching, ...",0.001465,0.563208,5,4,1.00,neutral,"Nobody is talking about life after launching, ...","Nobody is talking about life after launching, ...",Nobody is talking about life after launching b...,Nobody is talking about life after launching b...,Nobody is talking about life after launching b...,nobody is talking about life after launching b...,nobody talking life launching everyone glorifi...
1,Time for a quick progress,Time for a quick progress October is nearly go...,0.000482,0.753155,4,2,1.00,neutral,Time for a quick progress October is nearly go...,Time for a quick progress October is nearly go...,Time for a quick progress October is nearly go...,Time for a quick progress October is nearly go...,Time for a quick progress October is nearly go...,time for a quick progress october is nearly go...,time quick progress october nearly gone share ...
2,I built a SaaS for passive income… now I’m mor...,I built a SaaS for passive income… now I’m mor...,0.008917,0.717628,8,7,0.89,bad,I built a SaaS for passive income… now I’m mor...,I built a SaaS for passive income… now I’m mor...,I built a SaaS for passive income… now I’m mor...,I built a SaaS for passive income… now I’m mor...,I built a SaaS for passive income now Im more ...,i built a saas for passive income now im more ...,built saas passive income im active ever dream...
3,How did you went from 10 to 100 paying customers?,How did you went from 10 to 100 paying custome...,0.012576,0.271430,5,7,0.90,bad,How did you went from 10 to 100 paying custome...,How did you went from 10 to 100 paying custome...,How did you went from 10 to 100 paying custome...,How did you went from to paying customers We...,How did you went from to paying customers We...,how did you went from to paying customers were...,went paying customers building growthos helpin...
4,Anyone ever regret selling too much equity too...,Anyone ever regret selling too much equity too...,0.009597,0.284256,7,1,0.67,bad,Anyone ever regret selling too much equity too...,Anyone ever regret selling too much equity too...,Anyone ever regret selling too much equity too...,Anyone ever regret selling too much equity too...,Anyone ever regret selling too much equity too...,anyone ever regret selling too much equity too...,anyone ever regret selling much equity early v...
5,After 3 months I finally got my first paying u...,After 3 months I finally got my first paying u...,0.064827,0.719796,4,2,0.75,bad,After 3 months I finally got my first paying u...,After 3 months I finally got my first paying u...,After 3 months I finally got my first paying u...,After months I finally got my first paying us...,After months I finally got my first paying us...,after months i finally got my first paying use...,months finally got first paying user today bui...
6,What are some of the weirdest ways people have...,What are some of the weirdest ways people have...,0.004708,0.368361,28,7,0.82,bad,What are some of the weirdest ways people have...,What are some of the weirdest ways people have...,What are some of the weirdest ways people have...,What are some of the weirdest ways people have...,What are some of the weirdest ways people have...,what are some of the weirdest ways people have...,weirdest ways people made decent amount income...
7,Business Advice,Business Advice I'm exploring a D2C fintech bu...,0.968228,0.333799,9,6,1.00,good,Business Advice I'm exploring a D2C fintech bu...,Business Advice I'm exploring a D2C fintech bu...,Business Advice Im exploring a D2C fintech bus...,Business Advice Im exploring a DC fintech busi...,Business Advice Im exploring a DC fintech busi...,business advice im exploring a dc fintech busi...,business advice im exploring dc fintech busine...
8

In [ ]:
def remove_stopwords(text_final):
    return ' '.join(word for word in text_final.split() if word not in stop_words)
df.head(10)

,title,text,post_sentiment,avg_comment_sentiment,num_comments,upvotes,upvote_ratio,label,text_no_url,text_no_emoji,text_no_punct,text_no_num,text_no_special,text_cleaned,text_final
0,"Nobody is talking about life after launching, ...","Nobody is talking about life after launching, ...",0.001465,0.563208,5,4,1.00,neutral,"Nobody is talking about life after launching, ...","Nobody is talking about life after launching, ...",Nobody is talking about life after launching b...,Nobody is talking about life after launching b...,Nobody is talking about life after launching b...,nobody is talking about life after launching b...,nobody talking life launching everyone glorifi...
1,Time for a quick progress,Time for a quick progress October is nearly go...,0.000482,0.753155,4,2,1.00,neutral,Time for a quick progress October is nearly go...,Time for a quick progress October is nearly go...,Time for a quick progress October is nearly go...,Time for a quick progress October is nearly go...,Time for a quick progress October is nearly go...,time for a quick progress october is nearly go...,time quick progress october nearly gone share ...
2,I built a SaaS for passive income… now I’m mor...,I built a SaaS for passive income… now I’m mor...,0.008917,0.717628,8,7,0.89,bad,I built a SaaS for passive income… now I’m mor...,I built a SaaS for passive income… now I’m mor...,I built a SaaS for passive income… now I’m mor...,I built a SaaS for passive income… now I’m mor...,I built a SaaS for passive income now Im more ...,i built a saas for passive income now im more ...,built saas passive income im active ever dream...
3,How did you went from 10 to 100 paying customers?,How did you went from 10 to 100 paying custome...,0.012576,0.271430,5,7,0.90,bad,How did you went from 10 to 100 paying custome...,How did you went from 10 to 100 paying custome...,How did you went from 10 to 100 paying custome...,How did you went from to paying customers We...,How did you went from to paying customers We...,how did you went from to paying customers were...,went paying customers building growthos helpin...
4,Anyone ever regret selling too much equity too...,Anyone ever regret selling too much equity too...,0.009597,0.284256,7,1,0.67,bad,Anyone ever regret selling too much equity too...,Anyone ever regret selling too much equity too...,Anyone ever regret selling too much equity too...,Anyone ever regret selling too much equity too...,Anyone ever regret selling too much equity too...,anyone ever regret selling too much equity too...,anyone ever regret selling much equity early v...
5,After 3 months I finally got my first paying u...,After 3 months I finally got my first paying u...,0.064827,0.719796,4,2,0.75,bad,After 3 months I finally got my first paying u...,After 3 months I finally got my first paying u...,After 3 months I finally got my first paying u...,After months I finally got my first paying us...,After months I finally got my first paying us...,after months i finally got my first paying use...,months finally got first paying user today bui...
6,What are some of the weirdest ways people have...,What are some of the weirdest ways people have...,0.004708,0.368361,28,7,0.82,bad,What are some of the weirdest ways people have...,What are some of the weirdest ways people have...,What are some of the weirdest ways people have...,What are some of the weirdest ways people have...,What are some of the weirdest ways people have...,what are some of the weirdest ways people have...,weirdest ways people made decent amount income...
7,Business Advice,Business Advice I'm exploring a D2C fintech bu...,0.968228,0.333799,9,6,1.00,good,Business Advice I'm exploring a D2C fintech bu...,Business Advice I'm exploring a D2C fintech bu...,Business Advice Im exploring a D2C fintech bus...,Business Advice Im exploring a DC fintech busi...,Business Advice Im exploring a DC fintech busi...,business advice im exploring a dc fintech busi...,business advice im exploring dc fintech busine...
8

In [ ]:
df.drop(['title', 'text', 'text_no_url','text_no_emoji','text_no_punct','text_no_num','text_no_special','text_cleaned','num_comments'], axis=1, inplace=True)
df.head(10)

,post_sentiment,avg_comment_sentiment,num_comments,upvotes,upvote_ratio,label,text_final
0,0.001465,0.563208,5,4,1.00,neutral,nobody talking life launching everyone glorifi...
1,0.000482,0.753155,4,2,1.00,neutral,time quick progress october nearly gone share ...
2,0.008917,0.717628,8,7,0.89,bad,built saas passive income im active ever dream...
3,0.012576,0.271430,5,7,0.90,bad,went paying customers building growthos helpin...
4,0.009597,0.284256,7,1,0.67,bad,anyone ever regret selling much equity early v...
5,0.064827,0.719796,4,2,0.75,bad,months finally got first paying user today bui...
6,0.004708,0.368361,28,7,0.82,bad,weirdest ways people made decent amount income...
7,0.968228,0.333799,9,6,1.00,good,business advice im exploring dc fintech busine...
8,0.598869,0.426221,7,21,0.93,good,budgeting ai tools worth hype bb business plan...
9,0.130740,0.345931,20,8,0.79,bad,watched series startup almost die nobody could...


In [3]:
import pandas as pd
import nltk
from nltk import word_tokenize
nltk.download('punkt')
df = pd.read_csv('E:\saas-idea-validator\data\processed\cleaned_dataset.csv')
df['token_text'] = df['text_final'].apply(lambda x: word_tokenize(str(x)))
print("Tokenized dataset saved as 'tokenized_dataset.csv'")
print(df.head())

[nltk_data] Downloading package punkt to C:\Users\TUF
[nltk_data]     GAMING\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Tokenized dataset saved as 'tokenized_dataset.csv'
   post_sentiment  avg_comment_sentiment  upvotes  upvote_ratio  \
0        0.001465               0.563208        4          1.00   
1        0.000482               0.753155        2          1.00   
2        0.008917               0.717628        7          0.89   
3        0.012576               0.271430        7          0.90   
4        0.009597               0.284256        1          0.67   

                                          text_final  label_numeric  \
0  nobody talking life launching everyone glorifi...              2   
1  time quick progress october nearly gone share ...              2   
2  built saas passive income im active ever dream...              0   
3  went paying customers building growthos helpin...              0   
4  anyone ever regret selling much equity early v...              0   

                                          token_text  
0  [nobody, talking, life, launching, everyone, g...  
1  [time,

In [6]:
df.drop(columns=['text_final'], inplace=True)
df.head(10)

,post_sentiment,avg_comment_sentiment,upvotes,upvote_ratio,label_numeric,token_text
0,0.001465,0.563208,4,1.00,2,"[nobody, talking, life, launching, everyone, g..."
1,0.000482,0.753155,2,1.00,2,"[time, quick, progress, october, nearly, gone,..."
2,0.008917,0.717628,7,0.89,0,"[built, saas, passive, income, im, active, eve..."
3,0.012576,0.271430,7,0.90,0,"[went, paying, customers, building, growthos, ..."
4,0.009597,0.284256,1,0.67,0,"[anyone, ever, regret, selling, much, equity, ..."
5,0.064827,0.719796,2,0.75,0,"[months, finally, got, first, paying, user, to..."
6,0.004708,0.368361,7,0.82,0,"[weirdest, ways, people, made, decent, amount,..."
7,0.968228,0.333799,6,1.00,1,"[business, advice, im, exploring, dc, fintech,..."
8,0.598869,0.426221,21,0.93,1,"[budgeting, ai, tools, worth, hype, bb, busine..."
9,0.130740,0.345931,8,0.79,0,"[watched, series, startup, almost, die, nobody..."


In [5]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
df = pd.read_csv(r'E:\saas-idea-validator\data\processed\tokenized_dataset.csv')
w2v_model = Word2Vec(sentences=df['token_text'], vector_size=100, window=5, min_count=1, workers=4)

# --- Step 5: Sentence vector (average of word vectors) ---
def get_sentence_vector(tokens):
    vectors = [w2v_model.wv[word] for word in tokens if word in w2v_model.wv]
    if len(vectors) == 0:
        return np.zeros(w2v_model.vector_size)
    return np.mean(vectors, axis=0)

df['vector'] = df['token_text'].apply(get_sentence_vector)
df.head(10)

,post_sentiment,avg_comment_sentiment,upvotes,upvote_ratio,label_numeric,token_text,vector
0,0.001465,0.563208,4,1.00,2,"['nobody', 'talking', 'life', 'launching', 'ev...","[0.09650341, 0.040028777, -0.10762745, 0.14013..."
1,0.000482,0.753155,2,1.00,2,"['time', 'quick', 'progress', 'october', 'near...","[0.053671077, 0.040345866, -0.1391077, 0.14981..."
2,0.008917,0.717628,7,0.89,0,"['built', 'saas', 'passive', 'income', 'im', '...","[0.08293684, 0.048011195, -0.12019755, 0.13608..."
3,0.012576,0.271430,7,0.90,0,"['went', 'paying', 'customers', 'building', 'g...","[0.074951984, 0.04325141, -0.11598554, 0.13827..."
4,0.009597,0.284256,1,0.67,0,"['anyone', 'ever', 'regret', 'selling', 'much'...","[0.092327476, 0.04218995, -0.12825231, 0.11741..."
5,0.064827,0.719796,2,0.75,0,"['months', 'finally', 'got', 'first', 'paying'...","[0.08120428, 0.03531731, -0.09799173, 0.138135..."
6,0.004708,0.368361,7,0.82,0,"['weirdest', 'ways', 'people', 'made', 'decent...","[0.0692007, 0.034971014, -0.14067842, 0.126213..."
7,0.968228,0.333799,6,1.00,1,"['business', 'advice', 'im', 'exploring', 'dc'...","[0.074771546, 0.06295054, -0.092025995, 0.1632..."
8,0.598869,0.426221,21,0.93,1,"['budgeting', 'ai', 'tools', 'worth', 'hype', ...","[0.0934959, 0.04018701, -0.11599435, 0.1403434..."
9,0.130740,0.345931,8,0.79,0,"['watched', 'series', 'startup', 'almost', 'di...","[0.09127291, 0.036548633, -0.108102396, 0.1432..."


In [6]:
# Save the dataframe to a new CSV file
df.to_csv('E:\\saas-idea-validator\\data\\processed\\vectorised_dataset.csv', index=False)

print("Vectorised dataset saved as 'E:\\saas-idea-validator\\data\\processed\\vectorised_dataset.csv'")


Vectorised dataset saved as 'E:\saas-idea-validator\data\processed\vectorised_dataset.csv'
